# News data collection

In [9]:
from src.utils import news_data

# All capital letters for the parameters
df = news_data.extract_sentiment(ticker='FOREX:EUR', date_from='20230404T0130', sort='LATEST')
# Save csv file to folder
df.to_csv(r'C:\Users\jessi\OneDrive\1 Projects\1-svts\data\external\news-analysis\eur_sentiment.csv', index=False)


        ticker relevance_score ticker_sentiment_score ticker_sentiment_label  \
0    FOREX:EUR         0.05654               0.051923                Neutral   
1    FOREX:EUR        0.382925                0.23771       Somewhat-Bullish   
2    FOREX:EUR        0.710813               0.391358                Bullish   
3    FOREX:EUR        0.029436               0.089079                Neutral   
4    FOREX:EUR        0.165288               0.152561       Somewhat-Bullish   
..         ...             ...                    ...                    ...   
761  FOREX:EUR        0.083833               0.016985                Neutral   
762  FOREX:EUR        0.836003               0.174806       Somewhat-Bullish   
763  FOREX:EUR        0.092927              -0.002171                Neutral   
764  FOREX:EUR        0.085022               0.074107                Neutral   
765  FOREX:EUR        0.143415                    0.0                Neutral   

          time_published(est)  
0   202